In [91]:
import re
from pathlib import Path

import numpy as np
import pandas as pd

In [92]:
data_path = Path("example.txt")
raw_data = data_path.read_text().splitlines()
print(raw_data)

['L68', 'L30', 'R48', 'L5', 'R60', 'L55', 'L1', 'L99', 'R14', 'L82']


In [93]:
pattern = re.compile(r"(L|R)(\d+)")

direction_list = [
    pattern.match(line).group(1) for line in raw_data if pattern.match(line)
]
direction_list = ["R", *direction_list]
direction = pd.Series(direction_list)
direction = pd.Categorical(direction, categories=["L", "R"])

distance_list = [
    int(pattern.match(line).group(2))
    for line in raw_data
    if pattern.match(line)
]
distance_list = [50, *distance_list]
distance = pd.Series(distance_list)

data = {"direction": direction, "distance": distance}
lock_df = pd.DataFrame(data)
lock_df

,direction,distance
0,R,50
1,L,68
2,L,30
3,R,48
4,L,5
5,R,60
6,L,55
7,L,1
8,L,99
9,R,14


In [94]:
ROLLOVER = 100

lock_df = lock_df.assign(
    rotation=lambda df: df.distance
    * df.direction.cat.codes.replace({0: -1, 1: 1}),
    position=lambda df: df.rotation.cumsum() % ROLLOVER,
    previous_position=lambda df: df.position.shift(1, fill_value=0),
    truncated_rotation=lambda df: np.where(
        df.rotation.abs() >= ROLLOVER,
        df.rotation % np.where(df.rotation >= 0, ROLLOVER, -ROLLOVER),
        df.rotation,
    ),
    full_rotations=lambda df: df.rotation.abs() // ROLLOVER,
    raw_position=lambda df: df.previous_position + df.truncated_rotation,
    centered=lambda df: df.position == 0,
    crossed=lambda df: (
        df.centered
        | (df.raw_position >= ROLLOVER)
        | ((df.raw_position < 0) & (df.previous_position != 0))
    ),
)

lock_df

,direction,distance,rotation,position,previous_position,truncated_rotation,full_rotations,raw_position,centered,crossed
0,R,50,50,50,0,50,0,50,False,False
1,L,68,-68,82,50,-68,0,-18,False,True
2,L,30,-30,52,82,-30,0,52,False,False
3,R,48,48,0,52,48,0,100,True,True
4,L,5,-5,95,0,-5,0,-5,False,False
5,R,60,60,55,95,60,0,155,False,True
6,L,55,-55,0,55,-55,0,0,True,True
7,L,1,-1,99,0,-1,0,-1,False,False
8,L,99,-99,0,99,-99,0,0,True,True
9,R,14,14,14,0,14,0,14,False,False


In [95]:
print(lock_df.centered.sum())

3


In [96]:
print(lock_df.crossed.sum() + lock_df.full_rotations.sum())

6


In [97]:
pos = 50
count = 0
for rot in lock_df.rotation[1:]:
    prev_pos = pos
    if abs(rot) >= ROLLOVER:
        count += abs(rot) // ROLLOVER
        trunc_rot = rot % (ROLLOVER if rot > 0 else -ROLLOVER)
    else:
        trunc_rot = rot
    pos += trunc_rot
    if (
        (pos == 0 and prev_pos != 0)
        or pos >= ROLLOVER
        or (pos < 0 and prev_pos != 0)
    ):
        count += 1
    pos %= ROLLOVER
    print(
        f"Rotation: {rot}, Position: {pos}, Count: {count}",
    )
count


Rotation: -68, Position: 82, Count: 1
Rotation: -30, Position: 52, Count: 1
Rotation: 48, Position: 0, Count: 2
Rotation: -5, Position: 95, Count: 2
Rotation: 60, Position: 55, Count: 3
Rotation: -55, Position: 0, Count: 4
Rotation: -1, Position: 99, Count: 4
Rotation: -99, Position: 0, Count: 5
Rotation: 14, Position: 14, Count: 5
Rotation: -82, Position: 32, Count: 6


6